<a href="https://colab.research.google.com/github/thegoodgamer14/tech-spec-extractor/blob/main/baseline_distilbert_conll2003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/thegoodgamer14/tech-spec-extractor.git
%cd tech-spec-extractor/notebooks

fatal: destination path 'tech-spec-extractor' already exists and is not an empty directory.
/content/tech-spec-extractor/notebooks


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/tech-spec-extractor')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
DATA_DIR = '/content/drive/MyDrive/tech-spec-extractor/data'
MODELS_DIR = '/content/drive/MyDrive/tech-spec-extractor/models'
RESULTS_DIR = '/content/drive/MyDrive/tech-spec-extractor/results'

In [5]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install numpy as np
!pip install pypdf2
!pip install seqeval
!pip install evaluate
!pip install huggingface_hub

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [6]:
import torch
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer
)
from huggingface_hub import notebook_login

notebook_login()

In [7]:
print("Loading CoNLL-2003 dataset...")
try:
    conll_dataset = load_dataset("conll2003")
    print("Dataset loaded successfully!")
except Exception as e:
    print(f"Error loading dataset: {e}")
    raise e

print("Dataset structure:")
print(conll_dataset)

label_list = conll_dataset["train"].features["ner_tags"].feature.names
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}
num_labels = len(label_list)

print(f"\nNER Tags: {label_list}")
print(f"Number of labels: {num_labels}")

print("\nSample from training data:")
sample = conll_dataset["train"][0]
print("Tokens:", sample["tokens"][:10])
print("NER tags:", [label_list[tag] for tag in sample["ner_tags"][:10]])

Loading CoNLL-2003 dataset...
The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset loaded successfully!
Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

NER Tags: ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
Number of labels: 9

Sample from training data:
Tokens: ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
NER tags: ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [8]:
label_list = conll_dataset["train"].features["ner_tags"].feature.names
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}
num_labels = len(label_list)

print(f"\nNER Tags: {label_list}")
print(f"Number of labels: {num_labels}")
print("\nSample from training data:")
sample = conll_dataset["train"][0]
print("Tokens:", sample["tokens"][:10])
print("NER tags:", [label_list[tag] for tag in sample["ner_tags"][:10]])


NER Tags: ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
Number of labels: 9

Sample from training data:
Tokens: ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
NER tags: ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [9]:
model_checkpoint = "distilbert-base-uncased"
print(f"Loading tokenizer and model for: {model_checkpoint}")

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id_to_label,
    label2id=label_to_id
)
print("Model and tokenizer loaded successfully.")

Loading tokenizer and model for: distilbert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model and tokenizer loaded successfully.


In [10]:
print("Setting up preprocessing function...")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

print("Applying preprocessing to the dataset...")
tokenized_datasets = conll_dataset.map(tokenize_and_align_labels, batched=True)
print("Preprocessing complete.")
print(f"Tokenized dataset structure: {tokenized_datasets}")

print("\nSample from preprocessed training data:")
sample = tokenized_datasets["train"][0]
print(f"Input IDs (first 10): {sample['input_ids'][:10]}")
print(f"Attention Mask (first 10): {sample['attention_mask'][:10]}")
print(f"Labels (first 10): {sample['labels'][:10]}")

Setting up preprocessing function...
Applying preprocessing to the dataset...


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Preprocessing complete.
Tokenized dataset structure: DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

Sample from preprocessed training data:
Input IDs (first 10): [101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012]
Attention Mask (first 10): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Labels (first 10): [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0]


In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
print("Data collator set up successfully.")

print("Setting up evaluation metrics using 'evaluate' and 'seqeval'...")
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
print("Evaluation metric function defined.")

print("Configuring training arguments...")
output_dir = "./results_baseline"

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs_baseline',
    logging_steps=100,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)
print("Training arguments configured.")

print("Instantiating Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
print("Trainer instantiated successfully.")

Data collator set up successfully.
Setting up evaluation metrics using 'evaluate' and 'seqeval'...
Evaluation metric function defined.
Configuring training arguments...
Training arguments configured.
Instantiating Trainer...


<ipython-input-13-cfea78f0acb4>:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Trainer instantiated successfully.


In [14]:
print("Starting baseline model fine-tuning...")
try:
    train_result = trainer.train()
    print("Training finished successfully!")

    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)

    print("\n--- Training Metrics ---")
    for key, value in metrics.items():
        print(f"{key}: {value:.4f}")

except Exception as e:
    print(f"\nAn error occurred during training: {e}")
    print("This might be due to GPU memory limits. Try reducing batch sizes.")
    raise e

trainer.save_model(f"{output_dir}/final_model")
tokenizer.save_pretrained(f"{output_dir}/final_model")
print(f"Baseline model saved to {output_dir}/final_model")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Starting baseline model fine-tuning...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aryanhs (aryanhs-indian-institute-of-information-technology-and-m) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.067600,0.053312,0.912705,0.927297,0.919943,0.984677
2,0.037200,0.051317,0.920053,0.941266,0.930538,0.986177
3,0.022300,0.051062,0.929310,0.938068,0.933668,0.986722


Training finished successfully!
***** train metrics *****
  epoch                    =        3.0
  total_flos               =   475088GF
  train_loss               =     0.0676
  train_runtime            = 0:06:48.56
  train_samples_per_second =      103.1
  train_steps_per_second   =      6.447

--- Training Metrics ---
train_runtime: 408.5650
train_samples_per_second: 103.1000
train_steps_per_second: 6.4470
total_flos: 510122266253334.0000
train_loss: 0.0676
epoch: 3.0000
Baseline model saved to ./results_baseline/final_model


In [15]:
print("Evaluating the fine-tuned baseline model on the test set...")
eval_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])

print("\n--- Baseline Model CoNLL-2003 Test Set Results ---")
print(f"Precision: {eval_results.get('eval_precision', 'N/A'):.4f}")
print(f"Recall:    {eval_results.get('eval_recall', 'N/A'):.4f}")
print(f"F1-Score:  {eval_results.get('eval_f1', 'N/A'):.4f}")
print(f"Accuracy:  {eval_results.get('eval_accuracy', 'N/A'):.4f}")
print("----------------------------------------------------")

trainer.log_metrics("eval_test", eval_results)
trainer.save_metrics("eval_test", eval_results)

print("Baseline model training and evaluation complete!")

Evaluating the fine-tuned baseline model on the test set...



--- Baseline Model CoNLL-2003 Test Set Results ---
Precision: 0.8862
Recall:    0.9003
F1-Score:  0.8932
Accuracy:  0.9789
----------------------------------------------------
***** eval_test metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.9789
  eval_f1                 =     0.8932
  eval_loss               =     0.1159
  eval_precision          =     0.8862
  eval_recall             =     0.9003
  eval_runtime            = 0:00:04.89
  eval_samples_per_second =    705.467
  eval_steps_per_second   =      44.13
Baseline model training and evaluation complete!
